In [1]:
import sys
import os
import keras_tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

2025-10-04 16:02:34.641416: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-04 16:02:40.934451: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-04 16:02:40.934780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-04 16:02:40.937503: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-04 16:02:40.937768: I external/local_xla/xla/stream_executor

Epoch 1/200


I0000 00:00:1759586575.025113    5215 service.cc:145] XLA service 0xfffe60004080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759586575.025200    5215 service.cc:153]   StreamExecutor device (0): Orin, Compute Capability 8.7
2025-10-04 16:02:55.342630: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-04 16:02:55.816072: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90300


28/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.7338

I0000 00:00:1759586578.026553    5215 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - loss: 22.0289 - val_loss: 19.7420
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 16.4591 - val_loss: 16.7948
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.6923 - val_loss: 15.1357
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.8625 - val_loss: 12.9380
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.5380 - val_loss: 10.2294
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.6751 - val_loss: 6.9730
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.7022 - val_loss: 3.9012
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4365 - val_loss: 1.9307
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2666 - val_loss: 1.1224
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8523 - val_loss: 0.8720
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6956 - val_loss: 0.7375
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.61

/home/capybara/Desktop/matura_project_python/github/matura/FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


0 + -3 + -3
2543
-6.0

Expressions not in x:
-1 + 2 - 1
True
1457
0.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [4]:
input_shape = x_train[0].shape
def build_model(hp, input_shape):
    num_neurons = hp.Int("num_neurons", 1, 256)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")
    lil_model = keras.Sequential()
    lil_model.add(keras.Input(shape=input_shape))
    lil_model.add(layers.Flatten())
    for i in range(num_layers):
        lil_model.add(layers.Dense(num_neurons)),
        lil_model.add(PReLU())
        if dropoutTF == True:
            lil_model.add(layers.Dropout(0.1))
    lil_model.add(layers.Dense(1, activation='linear'))
    lil_model.compile(optimizer="adam", loss="mse")
    return lil_model
build_model(keras_tuner.HyperParameters(), input_shape)

<Sequential name=sequential_1, built=True>

In [5]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="FNNTuner",
    project_name="tuner",
)

In [6]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [early_stopping])

Trial 100 Complete [00h 00m 48s]
val_loss: 0.04782472923398018

Best val_loss So Far: 0.002553966362029314
Total elapsed time: 01h 49m 40s


In [12]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_3 (PReLU)               │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,609 (18.00 KB)

 Trainable params: 4,609 (18.00 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in FNNTuner/tuner
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 094 summary
Hyperparameters:
num_neurons: 256
num_layers: 1
dropoutT/F: False
Score: 0.002553966362029314

Trial 013 summary
Hyperparameters:
num_neurons: 220
num_layers: 1
dropoutT/F: False
Score: 0.0025585999246686697

Trial 011 summary
Hyperparameters:
num_neurons: 185
num_layers: 1
dropoutT/F: False
Score: 0.0026173184160143137

Trial 014 summary
Hyperparameters:
num_neurons: 203
num_layers: 1
dropoutT/F: False
Score: 0.0029141725972294807

Trial 015 summary
Hyperparameters:
num_neurons: 158
num_layers: 1
dropoutT/F: False
Score: 0.0031476537697017193

Trial 009 summary
Hyperparameters:
num_neurons: 256
num_layers: 2
dropoutT/F: False
Score: 0.008398172445595264

Trial 002 summary
Hyperparameters:
num_neurons: 238
num_layers: 8
dropoutT/F: False
Score: 0.011507810093462467

Trial 003 summary
Hyperparameters:
num_neurons: 107
num_layers: 2
dropoutT/F: False
Score: 0.01

In [8]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [9]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [13]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  4.0600996
MRE in Range:  0.96254706
MAE longer Expressions:  5.259189
MAE out Range:  9.756357
MRE out Range:  1.0007672


In [14]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [15]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [16]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

0.0005047978320559031
0.022369005711986162
0.16265488204760103
0.009793222478758525
Benchmark: 0.19532190807040162
